In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from collections import OrderedDict
import seaborn as sns
from pandas.tools import plotting
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from collections import Counter
from statistics import mode

In [3]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
gender = pd.read_csv('gender_submission.csv')

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [6]:
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [7]:
train['Age'] =train['Age'].fillna(value=train['Age'].mean()) #menampilkan df2 dan mengisi A yg nan dengan mean


In [8]:
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [9]:
mode(train['Embarked'])
train['Embarked'] =train['Embarked'].fillna(value='S') #menampilkan df2 dan mengisi A yg nan dengan mean


In [11]:
train['Famnum']=train['SibSp']+train['Parch']
train['IsAlone']=np.where(train['Famnum']==0,1,0)
test['Famnum']=test['SibSp']+test['Parch']
test['IsAlone']=np.where(test['Famnum']==0,1,0)

In [12]:
train['Cabin'] =train['Cabin'].fillna(value=train['Cabin'].mode()[0])

In [13]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Famnum,IsAlone
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,B96 B98,S,1,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1,0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,B96 B98,S,0,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1,0
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,B96 B98,S,0,1


In [14]:
X = train.drop(['Survived','Name','Ticket','Cabin'],axis =1)
Y = train['Survived']

In [15]:
X.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Famnum,IsAlone
0,1,3,male,22.0,1,0,7.2500,S,1,0
1,2,1,female,38.0,1,0,71.2833,C,1,0
2,3,3,female,26.0,0,0,7.9250,S,0,1
3,4,1,female,35.0,1,0,53.1000,S,1,0
4,5,3,male,35.0,0,0,8.0500,S,0,1


In [16]:
X = pd.get_dummies(X, columns = ["Pclass","Sex","Embarked"], drop_first=True)

In [25]:
X.head()

,Age,SibSp,Parch,Fare,Famnum,IsAlone,Pclass_2,Pclass_3,Sex_male,Embarked_Q,Embarked_S
0,22.0,1,0,7.2500,1,0,0,1,1,0,1
1,38.0,1,0,71.2833,1,0,0,0,0,0,0
2,26.0,0,0,7.9250,0,1,0,1,0,0,1
3,35.0,1,0,53.1000,1,0,0,0,0,0,1
4,35.0,0,0,8.0500,0,1,0,1,1,0,1


In [30]:
X = X.drop(['SibSp'],axis =1)

In [31]:
train_X, test_X, train_y, test_y = train_test_split(X,Y,test_size = 0.2, random_state = 123)

In [95]:
decisiontree = DecisionTreeClassifier(min_samples_split=15, min_samples_leaf=10,random_state=123)
decisiontree.fit(train_X, train_y)
decisiontreepred = decisiontree.predict(test_X)
#pd.DataFrame(decisiontreepred)

In [96]:
cm1 = confusion_matrix(test_y,decisiontreepred)
sensitivity1 = cm1[1,1] / (cm1[1,0] + cm1[1,1])
specificity1 = cm1[0,0] / (cm1[0,0] + cm1[0,1])

In [97]:
print('Confusion Matrix : \n', cm1)
#print ('Accuracy :', accuracy1)
print('Sensitivy :', sensitivity1)
print('Specificity :', specificity1)
accuracy_score(test_y,decisiontreepred)

Confusion Matrix : 
 [[100  14]
 [ 14  51]]
Sensitivy : 0.7846153846153846
Specificity : 0.8771929824561403


0.8435754189944135

In [43]:
X.head()

,Age,Parch,Fare,Famnum,IsAlone,Pclass_2,Pclass_3,Sex_male,Embarked_Q,Embarked_S
0,22.0,0,7.2500,1,0,0,1,1,0,1
1,38.0,0,71.2833,1,0,0,0,0,0,0
2,26.0,0,7.9250,0,1,0,1,0,0,1
3,35.0,0,53.1000,1,0,0,0,0,0,1
4,35.0,0,8.0500,0,1,0,1,1,0,1


In [44]:
xtest = test.drop(['PassengerId','Name','Ticket','Cabin','SibSp'],axis =1)
xtest.head()

,Pclass,Sex,Age,Parch,Fare,Embarked,Famnum,IsAlone
0,3,male,34.5,0,7.8292,Q,0,1
1,3,female,47.0,0,7.0000,S,1,0
2,2,male,62.0,0,9.6875,Q,0,1
3,3,male,27.0,0,8.6625,S,0,1
4,3,female,22.0,1,12.2875,S,2,0


In [45]:
#xtest = pd.get_dummies(xtest, columns = ["Pclass","Sex","Embarked"], drop_first=True)
xtest.isnull().sum()

Pclass       0
Sex          0
Age         86
Parch        0
Fare         1
Embarked     0
Famnum       0
IsAlone      0
dtype: int64

In [46]:
xtest['Age'] =xtest['Age'].fillna(value=xtest['Age'].mean())

In [47]:
xtest['Fare'] =xtest['Fare'].fillna(value=xtest['Fare'].mean())

In [52]:
xtest = pd.get_dummies(xtest, columns = ["Pclass","Sex","Embarked"], drop_first=True)
xtest.isnull().sum()

Age           0
Parch         0
Fare          0
Famnum        0
IsAlone       0
Pclass_2      0
Pclass_3      0
Sex_male      0
Embarked_Q    0
Embarked_S    0
dtype: int64

In [53]:
dtpred1 = decisiontree.predict(xtest)
dtpred1 = pd.DataFrame(dtpred1)
dtpred1.columns = ['Survived']
dtpred1.head()

,Survived
0,0
1,0
2,1
3,0
4,1


In [54]:
hasil = pd.concat([test,dtpred1], axis=1)

In [55]:
hasil.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Famnum,IsAlone,Survived
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,0,1,0
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,1,0,0
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,0,1,1
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,0,1,0
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,2,0,1


In [113]:
hasil.to_csv('C:/Users/nourma059258/Documents/astra/06-06-18 Supervised Learning/titanic/mypredict.csv', index=False, header=True)

In [56]:
hasil1 = hasil[['PassengerId','Survived']]

In [58]:
hasil1.to_csv('C:/Users/nourma059258/Documents/astra/06-06-18 Supervised Learning/titanic/mypredict2.csv', index=False, header=True)